# ARTS2
Summary of [ARTS2](link) results from project: `[{{ project().name }}]` 

## Description
Antibiotic Resistant Target Seeker

In [ ]:
import pandas as pd
from pathlib import Path
import plotly.graph_objects as go

import altair as alt
import ast

import networkx as nx
#! pip install pygraphviz

import warnings
warnings.filterwarnings('ignore')

from IPython.display import display, Markdown, HTML
from itables import to_html_datatable as DT
import itables.options as opt
opt.css = """
.itables table td { font-style: italic; font-size: .8em;}
.itables table th { font-style: oblique; font-size: .8em; }
"""
opt.classes = ["display", "compact"]
opt.lengthMenu = [5, 10, 20, 50, 100, 200, 500]

In [ ]:
report_dir = Path("../")
df_arts = pd.read_csv(report_dir / "tables/df_arts_as-6.1.1.csv")

In [ ]:
result = {}
ctr = 1
for i in df_arts.index:
    hits = ast.literal_eval(df_arts.loc[i, "Genelist"])
    bgc_id = df_arts.loc[i, "bgc_id"]
    for num, hits in enumerate(hits):
        assert len(hits) == 7
        arts_hits = {'arts_id' : int(hits[0]),
                     'genome_id' : df_arts.loc[i, "genome_id"],
                     'bgc_id' : bgc_id,
                     'bgc_type' : df_arts.loc[i, "Type"],
                     'profile' : hits[1],
                     'start' : int(hits[2]),
                     'stop' : int(hits[3]),
                     'hits_type' : hits[4],
                     'description' : hits[5],
                     'function' : hits[6]
                    }
        result[ctr] = arts_hits
        ctr = ctr + 1
df_hits = pd.DataFrame.from_dict(result).T

display(HTML(DT(df_hits, columnDefs=[{"className": "dt-center", "targets": "_all"}],)))

In [ ]:
color = ["#264653", "#287271", "#2a9d8f", "#8ab17d", "#e9c46a", "#f4a261", "#ee8959", "#e76f51"]
function_map = df_hits.function.value_counts().to_dict()
arts_function_color_map = {}
for num, item in enumerate(function_map.keys()):
    if num+1 > len(color):
        arts_function_color_map[item] = 'grey'
    else:
        arts_function_color_map[item] = color[num]
arts_function_color_map['unclassified'] = "grey"

arts_node_mapping = df_hits.loc[:, ["profile", "description", "function"]].drop_duplicates().set_index("profile", drop=False)
bgc_id_mapping = df_hits.loc[:, ["bgc_id", "bgc_type", "genome_id"]].drop_duplicates().set_index("bgc_id", drop=False)#.T.to_dict()

for c in ["bgc_id", "bgc_type", "genome_id"]:
    arts_node_mapping[c] = None

for c in ["profile", "description", "function"]:
    bgc_id_mapping[c] = None
    
arts_node_mapping = arts_node_mapping.T.to_dict()
bgc_id_mapping = bgc_id_mapping.T.to_dict()

In [ ]:
color_map = []
G = nx.from_pandas_edgelist(df_hits, source='bgc_id', target='profile')

pos = nx.nx_agraph.graphviz_layout(G)

for g in G.nodes:
    if g in arts_node_mapping.keys():
        for column in arts_node_mapping[g].keys():
            attrib = str(column)
            G.nodes[g][attrib] = arts_node_mapping[g][attrib]
        color = arts_function_color_map[G.nodes[g]['function']]
        color_map.append(color)
        G.nodes[g]["color"] = color
        G.nodes[g]["node_type"] = "arts_model"
    elif g in bgc_id_mapping.keys():
        for column in bgc_id_mapping[g].keys():
            attrib = str(column)
            G.nodes[g][attrib] = bgc_id_mapping[g][attrib]
        color = "blue"
        color_map.append(color)
        G.nodes[g]["color"] = color
        G.nodes[g]["node_type"] = "bgc_region"

In [ ]:
for n, p in pos.items():
    G.nodes[n]['pos'] = p

In [ ]:
edge_trace = go.Scatter(
    x=[],
    y=[],
    line=dict(width=0.5,color='#888'),
    hoverinfo='none',
    mode='lines')

for edge in G.edges():
    x0, y0 = G.nodes[edge[0]]['pos']
    x1, y1 = G.nodes[edge[1]]['pos']
    edge_trace['x'] += tuple([x0, x1, None])
    edge_trace['y'] += tuple([y0, y1, None])

In [ ]:
node_trace = go.Scatter(
    x=[],
    y=[],
    text=[],
    mode='markers',
    hoverinfo='text',
    marker=dict(
        showscale=False,
        color=color_map,
        size=10,
        line=dict(width=0)))

for node in G.nodes():
    x, y = G.nodes[node]['pos']
    node_trace['x'] += tuple([x])
    node_trace['y'] += tuple([y])

In [ ]:
for node, adjacencies in enumerate(G.adjacency()):
    node_trace['marker']['color']+=tuple([len(adjacencies[1])])
    node_type = G.nodes[adjacencies[0]]['node_type']
    if node_type == "bgc_region":
        description = G.nodes[adjacencies[0]]['bgc_type']
        function = ""
    elif node_type == "arts_model":
        description = G.nodes[adjacencies[0]]['description']
        function = G.nodes[adjacencies[0]]['function']
    node_info = f"{adjacencies[0]}<br>{G.nodes[adjacencies[0]]['node_type']}<br>{description}<br>{function}"
    node_trace['text']+=tuple([node_info])

In [ ]:
fig = go.Figure(data=[edge_trace, node_trace],
             layout=go.Layout(
                #title='<br>AT&T network connections',
                #titlefont=dict(size=16),
                showlegend=False,
                hovermode='closest',
                margin=dict(b=20,l=5,r=5,t=40),
                xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                yaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                             width=600, height=600))

fig

In [ ]:
for node in G.nodes:
    for attrib in G.nodes[node]:
        if type(G.nodes[node][attrib]) == tuple:
            G.nodes[node][attrib] = str(G.nodes[node][attrib])
        elif G.nodes[node][attrib] == None:
            G.nodes[node][attrib] = str(G.nodes[node][attrib])

nx.write_graphml(G, "assets/data/arts.graphml")

## References

<font size="2">

{% for i in project().rule_used['arts']['references'] %}
- *{{ i }}*
{% endfor %}

</font>